# Import packages

In [1]:
import json
from chains.common import CommonChain
from chains.program import ProgramChain
from chains.course import CourseChain
from chains.default import DefaultChain
from embedding import initialize_embeddings
from models import initialize_llm
from router import PromptRouter
from utils import to_markdown

# Initialize models

In [2]:
llm = initialize_llm("gpt4")
embeddings = initialize_embeddings()

# Initialize Chains 

In [3]:
chain_info = json.load(open("chains/chain_info.json"))

chains = {
    "financing_a_stevens_education": CommonChain(
        "financing_a_stevens_education", embeddings
    ),
    "tuition_fees_and_other_expenses_for_undergraduate_students": CommonChain(
        "tuition_fees_and_other_expenses_for_undergraduate_students", embeddings
    ),
    "tuition_fees_and_other_expenses_for_graduate_students": CommonChain(
        "tuition_fees_and_other_expenses_for_graduate_students", embeddings
    ),
    "student_life_at_stevens": CommonChain("student_life_at_stevens", embeddings),
    "student_services_at_stevens": CommonChain(
        "student_services_at_stevens", embeddings
    ),
    "graduate_education_at_stevens": CommonChain(
        "graduate_education_at_stevens", embeddings
    ),
    "undergraduate_education_at_stevens": CommonChain(
        "undergraduate_education_at_stevens", embeddings
    ),
    "program_info_at_stevens": ProgramChain("department_info", embeddings),
    "off_campus_employment": CommonChain("off_campus_employment", embeddings),
    "course_info": CourseChain("course_info", embeddings),
}

print(chains["course_info"].vectorstore._collection.count())

5778


# Initialize Prompt Router

In [4]:
router = PromptRouter(llm=llm, chain_info=chain_info)

In [5]:
prompt = "What are the core courses to take in MS in Machine Learning?" #This can vary based on user input

# Route to correct chain

In [6]:
chain_name = router.get_chain_name(prompt=prompt)
print("Selected Chain Name:", chain_name)

Selected Chain Name: program_info_at_stevens


# Send the prompt via the chain to generate the output

In [7]:
curr_chain = chains.get(chain_name, DefaultChain())

response = curr_chain.process_prompt(llm, prompt=prompt)
to_markdown(response["answer"])

> The core courses to take in the MS in Machine Learning at Stevens Institute of Technology are as follows:
> 
> | Course Number | Course Title                                      | Credits |
> |---------------|---------------------------------------------------|---------|
> | CS 541        | Artificial Intelligence                           | 3       |
> | CS 559        | Machine Learning: Fundamentals and Applications   | 3       |
> | CS 560        | Statistical Machine Learning                      | 3       |
> | CS 583        | Deep Learning                                     | 3       |
> | CS 584        | Natural Language Processing                       | 3       |
> 
> Students must complete four out of the five courses listed above.